In [ ]:
#There is a really interesting dataset of genes that confer resistance to antibiotics.  
## 유전자-항생제 저항성 나타내는 데이터셋
#In the header is the gene ID and also the antibiotic ID at the end:  https://figshare.com/articles/dataset/COALA_datasets/11413302
## header:유전자ID, end:항생제ID  

In [10]:
from Bio import SeqIO

# Open the FASTA file
sequences = []
labels = []
with open("coala40.fa", "r") as handle:
    # Use SeqIO to parse the file and return a SeqRecord object
    i=0
    for record in SeqIO.parse(handle, "fasta"):
        # Print the ID and sequence of each record
        print(i+1," . ")
        print(record)
        antibiotic = record.description.split("|")[-1]
        print(antibiotic)
        print(record.id)
        resistence_gene = record.id.split("|")[0]
        print(resistence_gene)
        labels.append(resistence_gene)
        print(record.seq)
        sequences.append(record.seq)
        i += 1
        if i == 5: break

1  . 
ID: EBP77072.1|AFAM001-1|
Name: EBP77072.1|AFAM001-1|
Description: EBP77072.1|AFAM001-1| hypothetical protein GOS_7857327, partial [marine metagenome]|BLDB|BETA-LACTAM
Number of features: 0
Seq('MCGRRFRLSAAWLRWRAWSLPCAAERLHRWSRPENRSSQSGTLWLERQRDARKA...AVA')
BETA-LACTAM
EBP77072.1|AFAM001-1|
EBP77072.1
MCGRRFRLSAAWLRWRAWSLPCAAERLHRWSRPENRSSQSGTLWLERQRDARKAGNMPLNLRPRRTLACLAAFAVSACVGTVVPASVPPQTRSPTVSPRPVPAPAPAQREDRKPLTLPASLDPGFARPPPELNESINALWKAFPGKTGIAVARIDGDWQLAWRENDLFPQQSVSKLWVALAVLDAVDRGDVKLDDQVRIGREDLTLFSQPIAARVLAEGSINPTVAELLEQAVNRSDNTANDSLQRRIGGPQKVRDFIARKKLGLIRFGPGERLLQAQIAGLTWDQSMSLNRGFEAARSKLPREVRQAAMDRYLADPIDGMSPGAAARALTRLARGELLSAESTKYALDLLERTRTGPNRLRAGLPADWRWGHKTGTGQNLPPITAGYNDIGIATAPDGTRYAIVVMLADTTASVPARMQLMQAVA
2  . 
ID: WP_122865681.1|AFAM006-1|
Name: WP_122865681.1|AFAM006-1|
Description: WP_122865681.1|AFAM006-1| class A beta-lactamase [Listeria thailandensis]|BLDB|BETA-LACTAM
Number of features: 0
Seq('MGEMVFMKQKTIMRWGVLVLSVLVFSIAAWEISENRSSEKELKAEATPQINSVN...LAL')
BE

In [ ]:
#Can you use the COALA40.fa dataset to predict antibiotic from the gene sequence.  
## 유전자, 항생제 태그가 있고 / 유전자 서열이 있음  
### 유전자 서열로부터 항상제 예측하는 거 예측할 수 있냐?

#One idea we have is to use a pretrained protein model, like ProtBert or ProtT5-XL-UniRef50, 
## 하나의 아이디어는 사전학습된 단백질 모델을 이용하는 것- ProtBert 또는 ProtT5-XL-UniRef50 같은  
#to embed these proteins for a representation that may be easier to train a classifier 
## 이 단백질 임베딩 사용하면 분류 학습 쉬울 것  
#to predict the antibiotics that the gene is resistant to.  
#Like using these pretrained embeddings:  https://github.com/agemagician/ProtTrans
## 이 프레임웍 사용하면 될듯-모델 제공해주는


In [ ]:
#Like I'm finding that people are trying to make it easier to compute the embeddings like this package: 
#https://pypi.org/project/bio-transformers/
## 패키지도 있음, 임베딩 사용 관련한

#The challenge then would be how to use the embeddings to train a machine learning classifier to classify the type of antibiotic.  
#You could take a standard ML classifier or neural network.


#The papers you find, your thought process... all is very helpful even if you can't fully figure it out.

구현..?

# 관련 라이브러리 설치

In [1]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 라이브러리 임포트

In [3]:
from Bio import SeqIO

In [4]:
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load ProtBERT tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False)


# 데이터 읽기 및 전처리

In [5]:
from Bio import SeqIO

# Open the FASTA file
sequences = []
labels = []
with open("coala40.fa", "r") as handle:
    # Use SeqIO to parse the file and return a SeqRecord object
    for record in SeqIO.parse(handle, "fasta"):
        #resistence_gene = record.id.split("|")[0]
        antibiotic = record.description.split("|")[-1]
        labels.append(antibiotic)
        sequences.append(str(record.seq))
        

In [6]:
print( f'unique num: {len(set(labels))} / unique set: {set(labels)}' )

unique num: 14 / unique set: {'TETRACYCLINE', 'AMINOGLYCOSIDE', 'QUINOLONE', 'GLYCOPEPTIDE', 'BETA-LACTAM', 'PHENICOL', 'SULFONAMIDE', 'FOLATE-SYNTHESIS-INHABITOR', 'MULTIDRUG', 'FOSFOMYCIN', 'TRIMETHOPRIM', 'MACROLIDE', 'MACROLIDE/LINCOSAMIDE/STREPTOGRAMIN', 'BACITRACIN'}


In [7]:
from sklearn.preprocessing import LabelEncoder

# Convert string labels to numerical labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
print( f'unique num: {len(set(labels))} / unique set: {set(labels)}' )

unique num: 14 / unique set: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}


In [8]:
from sklearn.model_selection import train_test_split

# split the data into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(
    sequences, labels, test_size=0.2, random_state=42)

# split the training data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42)

# print the shapes of the resulting datasets
print("Training data shape:", len(train_data) )
print("Training labels shape:", len(train_labels) )
print("Validation data shape:", len(val_data) )
print("Validation labels shape:", len(val_labels) )
print("Test data shape:", len(test_data) )
print("Test labels shape:", len(test_labels) )

Training data shape: 3284
Training labels shape: 3284
Validation data shape: 822
Validation labels shape: 822
Test data shape: 1027
Test labels shape: 1027


In [9]:
train_encodings = tokenizer(train_data, truncation=True, padding=True)
val_encodings = tokenizer(val_data, truncation=True, padding=True)
test_encodings = tokenizer(test_data, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
class KoalaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = KoalaDataset(train_encodings, train_labels)
val_dataset = KoalaDataset(val_encodings, val_labels)
test_dataset = KoalaDataset(test_encodings, test_labels)

# 모델 작성 및 학습

In [11]:
from transformers import BertConfig, BertForPreTraining, BertTokenizer

In [ ]:
# define the configuration for the ProtBERT model
config = BertConfig.from_pretrained("Rostlab/prot_bert", output_hidden_states=True, num_labels=14)


# define the ProtBERT model
model = AutoModelForSequenceClassification.from_pretrained('Rostlab/prot_bert', num_labels=14)
#model = AutoModelForSequenceClassification.from_pretrained('Rostlab/prot_bert_bfd', num_labels=2)
#model = BertForPreTraining.from_pretrained("Rostlab/prot_bert", config=config)


# define the model configuration
#config = BertConfig.from_pretrained('Rostlab/prot_bert_bfd', num_labels=2)

# create the Trainer instance and train the model
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=1000,
    num_train_epochs=50, #5,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    #metric_for_best_model='accuracy',
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)
trainer.train()

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

Step,Training Loss,Validation Loss
100,2.458300,2.098892
200,1.914200,1.842072
300,1.848400,1.826129
400,1.842300,1.830972
500,1.867300,1.814874
600,1.868400,1.827572
700,1.865600,1.822829
800,1.849200,1.851566
900,1.825900,1.824914
1000,1.873500,1.822142


TrainOutput(global_step=10300, training_loss=1.853629790370904, metrics={'train_runtime': 3213.1107, 'train_samples_per_second': 51.103, 'train_steps_per_second': 3.206, 'total_flos': 1120032653320800.0, 'train_loss': 1.853629790370904, 'epoch': 50.0})

In [ ]:
# In this code, we first import the necessary libraries including the ProtBERT tokenizer and model for sequence classification. 
# We then load the Koala dataset and preprocess the sequences using the tokenizer. 
# The encoded sequences are then converted to PyTorch tensors along with their corresponding labels. 
# We split the dataset into train and test sets and define the training arguments and Trainer object. 
# We then train the model, evaluate it on the test set, and save it for future use.

# Note that this code assumes that you have already installed the necessary libraries and have downloaded the ProtBERT and Koala dataset. 
# You may need to modify the code to fit your specific use case. 
# Also, I have assumed that the labels in the Koala dataset are binary (0 or 1), but you may need to modify the code if your labels are multi-class.

# 평가

In [13]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from datasets import load_metric
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
metric= load_metric("accuracy")
test_dataloader = DataLoader(test_dataset, batch_size=128)
model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to("cuda") for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

100%|██████████| 9/9 [00:00<00:00, 10.54it/s]


{'accuracy': 0.3943524829600779}